In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET.Interactive,  2.0.0-preview.8"

#load "Parameters.fsx"
#load "UnixTime.fsx"
#load "AssetPrices.fsx"
#load "Graph.fsx"
#load "Drawdown.fsx"
#load "dEth.fsx"

open FSharp.Data
open FSharp.Stats

open Plotly.NET

open DEth
open Graph

open System.Collections.Generic

// drawChart
//     DEth.time
//     [
//         ("Actual", DEth.actualCollateral)
//         //("Predicted", predictedCollateral |> List.map (fun x -> x.Value))
//         ("Price Drawdown", DEth.predictedDrawdowns)
//         ("Collateral Drawdown", DEth.vaultDrawdowns)
//     ]

// drawMesh DEth.xOpt DEth.yOpt DEth.zOpt

drawSurface DEth.surface DEth.surfaceX DEth.surfaceY


Installed Packages FSharp.Data, 4.2.4 FSharp.Stats, 0.4.2 Plotly.NET.Interactive, 2.0.0-preview.8